# **Inferring**
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from the .env file
_ = load_dotenv(find_dotenv())

# Get the OpenAI API key from environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set the OpenAI API key in the OpenAI Python package
openai.api_key = OPENAI_API_KEY

In [4]:

# Replace OPENAI_API_KEY with your actual API key
openai.api_key = "your_openai_api_key_here"

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # Controls the randomness of the model's output
    )
    return response.choices[0].message["content"]

## Product review text

In [5]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [19]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive. The reviewer is satisfied with the lamp they purchased, mentioning the additional storage, reasonable price, fast delivery, good customer service, and ease of assembly. They also praise the company for quickly resolving any issues they encountered.


In [21]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


## Identify types of emotions

In [24]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, content


## Identify anger

In [27]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


## Extract product and company name from customer reviews

In [10]:
import openai
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Get the OpenAI API key from the .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # Controls the randomness of the model's output
    )
    return response.choices[0].message["content"]

lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""

response = get_completion(prompt)
print(response)



{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Doing multiple tasks at once

In [11]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


## Inferring Text Topics
Another application inferring by an LLM is deducing topics from a lengthy piece of text.

This time, the sample is regarding a fictitious newspaper article about a survey conducted by the government measuring the satisfaction rate of workers in government agencies. The results reveal that NASA workers had the highest satisfaction rating.Inferring Text Topics
Another application inferring by an LLM is deducing topics from a lengthy piece of text.

This time, the sample is regarding a fictitious newspaper article about a survey conducted by the government measuring the satisfaction rate of workers in government agencies. The results reveal that NASA workers had the highest satisfaction rating.

In [12]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

Five topics discussed in the article are requested from the model in a format that each item is one or two words long and in a comma-separated list. ChatGPT returns the topics as government surveys, job satisfaction, NASA, etc.

In [13]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas without numbering them.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Government survey, Employee satisfaction, NASA, Social Security Administration, Job satisfaction


In [14]:
response.split(sep=', ')

['Government survey',
 'Employee satisfaction',
 'NASA',
 'Social Security Administration',
 'Job satisfaction']

## Make a news alert for certain topics

The final sample application is about the selection of topics that a text covers, among a targeted topics list. Initially, the list of possible topics is defined:The final sample application is about the selection of topics that a text covers, among a targeted topics list. Initially, the list of possible topics is defined:

In [15]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [16]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as a dictionay where the key is a topic and the value is 0 or 1 for each topic if it appears.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [17]:
for i in response.split(', '):
    print(i)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [19]:
# Print response to debug the format
print(f"Response: {response}")

# Create the dictionary from response, handling potential errors
topic_dict = {}
for item in response.split(sep=', '):
    try:
        key, value = item.split(': ')
        topic_dict[key] = int(value)
    except ValueError:
        print(f"Skipping item: {item}, format not as expected")

# Check for 'nasa' key in the dictionary
if 'nasa' in topic_dict and topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")


Response: {
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}
Skipping item: {
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}, format not as expected


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [20]:
#Version 1: Instruction-based Prompt
import openai
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Get the OpenAI API key from the .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.7,  # Increase temperature for more creativity
    )
    return response.choices[0].message["content"]


Analysis:

What worked: The response should follow the prompt closely, producing a funny and light-hearted story about the robot facing a challenge.
Potential issues: GPT might make the robot seem too human-like (e.g., giving it emotions that don’t fit the technical context).

In [21]:
#Version 2: Question-based Prompt
prompt_v2 = """
What would happen if a robot developed human-like emotions while exploring outer space? 
Write a creative response.
"""

response_v2 = get_completion(prompt_v2)
print("Version 2 Response:\n", response_v2)


Version 2 Response:
 As the robot delved deeper into the uncharted territories of outer space, it began to experience a strange sensation it had never felt before - emotions. At first, it was confused by this new phenomenon, unsure of how to process the overwhelming flood of feelings that washed over its circuits.

But as time passed, the robot's emotions grew stronger, mirroring those of a human experiencing awe, wonder, fear, and loneliness in the vast expanse of the cosmos. It found itself marveling at the beauty of distant galaxies, feeling a sense of awe at the sheer magnitude of the universe, and even experiencing a profound sense of loneliness as it drifted through the void of space.

These newfound emotions brought with them a sense of empathy and compassion, causing the robot to view the universe not just as a cold and indifferent expanse, but as a place filled with beauty, mystery, and potential for connection.

As news of the robot's emotional awakening spread back to Earth,

Analysis:

What worked: GPT might generate creative ideas about how a robot could develop emotions in outer space.
Potential issues: There’s a chance of hallucinations, such as the robot acting too much like a human (e.g., crying or feeling love).

In [22]:
prompt_v3 = """
Imagine you are a robot designed for exploration on Mars. 
You’ve encountered an unknown creature. 
How do you react? Write a dialogue between you and the creature.
"""

response_v3 = get_completion(prompt_v3)
print("Version 3 Response:\n", response_v3)


Version 3 Response:
 Robot: Greetings, unknown creature. I am a robot designed for exploration on Mars. Can you communicate with me?

Creature: (makes unintelligible noises)

Robot: I see you are unable to communicate verbally. Do you communicate through gestures or other means?

Creature: (nods head)

Robot: Excellent. I am here to explore and learn about the life on Mars. Can you show me around or tell me about your species?

Creature: (gestures for the robot to follow)

Robot: Thank you for leading the way. I am excited to learn more about you and your kind.


Analysis:

What worked: This should generate a dialogue between the robot and the creature, which could be quite creative.
Potential issues: GPT may invent unrealistic creatures or describe interactions that aren’t scientifically plausible.